In [4]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [5]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agentic-RAG-Practice")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agentic-RAG-Practice


In [6]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

In [7]:
# 검색 결과를 가져옵니다.
search.invoke("새싹프로그램 강동캠퍼스 고객문의번호는 무엇인가요?")

[{'title': '강동구 - 강동구 added a new photo.',
  'url': 'https://www.facebook.com/100069164501223/photos/905877918394350/',
  'content': '👉 교육기간: 8월부터 내년 1월까지! ✅ 신청기간: 7월 27일까지 ✅ 신청: 새싹 홈페이지 (sesac.seoul.kr) ✅ 문의: 02-6249-1603, 1605 포트폴리오 완성부터 취업까지, 미래를 바꾸는 기회, 놓치지 마세요!',
  'score': 0.85421735},
 {'title': 'SeSAC 강동캠퍼스 - 청년취업사관학교',
  'url': 'https://sesac.seoul.kr/common/menu/html/900006004005/detail.do',
  'content': '주소 및 연락처. 서울특별시 강동구 고덕로 429 4층 강동캠퍼스: 02-6249-1601~2 sesac@sba.seoul.kr · 대중교통으로 오시는 방법. 5호선 강일역 1번출구 도보 30m.',
  'score': 0.82694983},
 {'title': '청년취업사관학교 새싹(SeSAC) 강동캠퍼스 AI과정 교육생 모집',
  'url': 'https://cmsfox.ewha.ac.kr/security/student/career-and-contest.do?mode=view&articleNo=726603&title=%EC%B2%AD%EB%85%84%EC%B7%A8%EC%97%85%EC%82%AC%EA%B4%80%ED%95%99%EA%B5%90+%EC%83%88%EC%8B%B9%28SeSAC%29+%EA%B0%95%EB%8F%99%EC%BA%A0%ED%8D%BC%EC%8A%A4+AI%EA%B3%BC%EC%A0%95+%EA%B5%90%EC%9C%A1%EC%83%9D+%EB%AA%A8%EC%A7%91',
  'content': '\\ 대상여부 확인 결과, 대상자가 아닌 경우 우대내용 적용하지 않음\n\n \n\n 다. 새싹

## 문서 기반 문서 검색 도구 Retriever

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/월간 SW 중심사회 7월호 이슈.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.ㅁ
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

In [11]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke("최근 바뀐 게임 변화에 대해 알려줘")

[Document(id='e200cf68-ef5d-4a67-9712-f225b81d9bda', metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2021-07-08T19:21:42+09:00', 'moddate': '2021-07-08T19:21:47+09:00', 'trapped': '/False', 'source': 'data/월간 SW 중심사회 7월호 이슈.pdf', 'total_pages': 24, 'page': 20, 'page_label': '78'}, page_content='[ 그림 15 ] 기술 발전에 따른 다양한 게임 분야의 진화\n▒ \x07특히 최근 클라우드·5G와 같은 기술들의 발전은 기존의 PC-모바일-콘솔 게임의 경계를 \n흐려지게 함\n◦\x07과거에는 고사양 게임을 하기 위해 게임 전용 PC를 맞추거나 콘솔 디바이스를 구입, 혹은 고성\n능 PC가 있는 장소(PC방 등)에 가서 게임을 해야 했음\n◦\x07하지만 클라우드 게임 분야의 발전으로 특정 게임을 위한 ‘최적의 게임 전용 디바이스’가 무의미해지고, 이로 인해 다양한 게임에 대한 진입장벽이 낮아짐\n◦\x07PC-모바일-콘솔 게임 분야의 경계가 흐려지면서 게임 개발에 제한이 없어졌을 뿐만 아니라 인공지능, VR·AR과 같은 최신 기술이 다양한 게임 분야에 적용되고 발전\n▒ \x07이는 게임이 단순 오락을 제공하는 콘텐츠에서 진화하여 다양한 가치가 창출되는 ‘플랫폼\n화’가 이루어지도록 촉진\n◦\x07특히 이러한 변화가 가장 활발히 이루어지고 있는 클라우드 게임 분야에는 기존 게임 개발·유통 기업뿐만 아니라 이동통신사·인터넷·IT제조 기업이 뛰어들어 생태계를 장악하려는 움직임을 보\n이고 있음\n78\n이슈\nISSUE'),
 Document(id='fbcc63da-b4d8-44a3-bf44-ef174956c441', met

#### 만든 retriever를 agent가 사용할 tool로 변경

In [12]:
from langchain.tools.retriever import create_retriever_tool
# 기존에 사용했던 retriever를 만들어주는 라이브러리
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

#### tools 정의

In [13]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

## Agent 생성

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [15]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

In [16]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Agent 실행

In [17]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [19]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "최근 바뀐 게임 변화에 대해 문서를 읽고 알려줘"}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
[도구 호출]
Tool: pdf_search
query: 최근 바뀐 게임 변화
Log: 
Invoking: `pdf_search` with `{'query': '최근 바뀐 게임 변화'}`




Invoking: `pdf_search` with `{'query': '최근 바뀐 게임 변화'}`


[ 그림 15 ] 기술 발전에 따른 다양한 게임 분야의 진화
▒ 특히 최근 클라우드·5G와 같은 기술들의 발전은 기존의 PC-모바일-콘솔 게임의 경계를 
흐려지게 함
◦과거에는 고사양 게임을 하기 위해 게임 전용 PC를 맞추거나 콘솔 디바이스를 구입, 혹은 고성
능 PC가 있는 장소(PC방 등)에 가서 게임을 해야 했음
◦하지만 클라우드 게임 분야의 발전으로 특정 게임을 위한 ‘최적의 게임 전용 디바이스’가 무의미해지고, 이로 인해 다양한 게임에 대한 진입장벽이 낮아짐
◦PC-모바일-콘솔 게임 분야의 경계가 흐려지면서 게임 개발에 제한이 없어졌을 뿐만 아니라 인공지능, VR·AR과 같은 최신 기술이 다양한 게임 분야에 적용되고 발전
▒ 이는 게임이 단순 오락을 제공하는 콘텐츠에서 진화하여 다양한 가치가 창출되는 ‘플랫폼
화’가 이루어지도록 촉진
◦특히 이러한 변화가 가장 활발히 이루어지고 있는 클라우드 게임 분야에는 기존 게임 개발·유통 기업뿐만 아니라 이동통신사·인터넷·IT제조 기업이 뛰어들어 생태계를 장악하려는 움직임을 보
이고 있음
78
이슈
ISSUE

게임의 진화 :
콘텐츠를 넘어 플랫폼으로
장지윤 
연구원jiyoonch@spri.kr
최혜리 
연구원hchoi@spri.krEvolution of Digital Game: From Contents to Platform
Executive Summary
기술의 발전과 사회적 변화가 가져온 ‘디지털 놀이’ 시대에 게임에 대한 새로운 정의의 필요성
이 대두되고 있다. 특히 디지털 네이티브인 MZ세대의 IT 기기 및 게임 이용행태는 이러한 변화 
동인을 지

### RunnableWithMessageHistory를 추가한 전체코드

In [21]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/월간 SW 중심사회 7월호 이슈.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의합니다 ##########

# Prompt 를 정의합니다 - 이 부분을 수정할 수 있습니다!
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의합니다 ##########

# 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": """최근 클라우드 게임과 메타버스의 확산은 기존 PC·모바일·콘솔 게임의 경계를 
     어떻게 흐리게 만들었으며, 그로 인해 게임 산업의 패러다임은 어떤 방향으로 전환되고 있나요? 문서의 내용으로 답해주세요. 
     문서에 해당 내용이 없다면 search해주세요."""},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 클라우드 게임과 메타버스의 확산
Log: 
Invoking: `pdf_search` with `{'query': '클라우드 게임과 메타버스의 확산'}`



[관찰 내용]
Observation: ▒ 5세대 초고속 통신망(5G)의 발달 등의 기술 진보는 클라우드 게임을 디바이스 종속적인 
게임 시장의 ‘게임 체인저’로 만드는 중
◦클라우드 방식의 게임 서비스를 의미하는 클라우드 게임은 게임용 디바이스의 컴퓨터 성능이 
아닌 클라우드 내에서 게임 연산이 처리되고 이용자는 이를 스트리밍 서비스로 받을 수 있는 
구조
◦클라우드 게임을 플레이하는 이용자들은 고사양 게임 콘텐츠를 위한 게임 전용 고사양 PC 및 
노트북 구입의 필요성이 없어진다는 것이 최대 장점
[ 그림 11 ] 다양한 기기 간 크로스 플레이를 지원하는 게임
* (좌) 포트나이트 ­ PC, 모바일, 콘솔 3사에서 전부 크로스 플레이를 지원, 
(우) 검은사막 ­ 국내 게임에서 진출을 꺼리는 콘솔 분야에서의 크로스 플레이를 지원
▒ 클라우드 게임 서비스에는 선점 효과를 기대하며 기존 게임 산업 뿐만 아니라 구글, 마이크
로소프트, 아마존, 텐센트, 이동통신사 등 다양한 IT 업계가 시장에 참여
◦게임 콘텐츠를 통해 클라우드 플랫폼과 네트워크, 데이터 센터 등을 선점하여 시장 내에서 다양
한 우월적 지위를 누릴 수 있기 때문
▒ 다양한 IT기업들의 참여를 비롯하여 디바이스 의존성 해소 등 클라우드 게임이 갖는 장점
들은 클라우드 게임 시장의 급격한 성장을 이끌 것으로 전망
◦뉴주(Newzoo)는 2021년 클라우드 게임 분야의 수익이 최초로 10억 달러를 돌파할 것으로 예상하
고 있으며, ’23년까지 전 세계 클라우드 게임 시장이 약 48억 달러 규모로 성장할 것을 예상24
24　Newzoo(2021.02.04.), ‘Newzoo’s Trends to Watch in 2021(Games, Esports and Mobile)’
73
MON

In [23]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
The recent expansion of cloud gaming and the metaverse is blurring the boundaries of traditional PC, mobile, and console games. Cloud gaming processes game computations within the cloud, allowing users to stream the service, thus eliminating the need to purchase high-spec PCs and laptops for high-end game content. This enables 'cross-play,' allowing the same game to be enjoyed on various devices, providing a gaming environment that is not restricted by platforms.

Additionally, the spread of the metaverse is transforming games from mere entertainment into a means of social interaction, contributing to the formation of new societies in virtual spaces. This evolution is turning games into platforms for creating new consumption opportunities or spaces for brand promotion.

These changes are shifting the paradigm of the gaming industry towards a "value shift to platform games," emphasizing the need for research that understands games as an organic ecosystem where various technologi

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

In [24]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "2024년 goty에 선정된 게임을 찾아주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024년 GOTY 선정 게임
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 GOTY 선정 게임'}`



[관찰 내용]
Observation: [{'title': 'Game Awards 2024: Astro Bot Triumphs, A Night Full of Awards', 'url': 'https://www.nacongaming.com/en-US/blog/game-awards-2024-results?srsltid=AfmBOopY6ZHJ5h8jgwXt9jvNxjMqKrVIQ7UTFBqR7hTjWBLRd5g2-Sh0', 'content': "The 10th edition of the Game Awards concluded on the night of December 12-13 at the Peacock Theater in Los Angeles, and the successor to Baldur’s Gate III has now been revealed. Astro Bot takes the crown with the prestigious title of Game of the Year 2024, marking an exceptional year for Team Asobi and Sony Interactive Entertainment. Here’s a recap of the awards given during this anniversary ceremony, hosted by the irreplaceable Geoff Keighley.\n\n## Game of the Year (GOTY)\n\nWinner: Astro Bot [...] The 2024 edition of the Game Awards was marked by Astro Bot’s dominance (4 awards including GOTY)

In [25]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "2023년은?"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2023 Game of the Year winner
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2023 Game of the Year winner'}`



[관찰 내용]
Observation: [{'title': 'The Game Awards 2023 Winners: The Full List - IGN', 'url': 'https://www.ign.com/articles/the-game-awards-2023-winners-the-full-list', 'content': "This category is 100% voted on by fans for their favorite game of 2023.\n\n Baldur's Gate 3 - WINNER\n Cyberpunk 2077: Phantom Liberty\n Genshin Impact\n Marvel's Spider-Man 2\n The Legend of Zelda: Tears of the Kingdom\n\n### The Game Awards 2023: Every Winner and Nominee\n\nBaldur's Gate 3 won Game of the Year at the TGAs. The first 18 games on this list all won an award -- the rest were nominated and came up empty.\n\nSee All [...] Not only did Baldur's Gate 3 win Game of the Year and shadow drop its Xbox version, but it also won Best Community Support, Best RPG, Best Multiplayer, and the Player's Voice category. Alan Wake 2 followed wi